<a href="https://colab.research.google.com/github/debernall/Proyecto/blob/main/Proyecto_bootcamp_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introducción

# Proyecto Lluvia

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
import os

In [3]:
import plotly.express as pex
from plotly.subplots import make_subplots
import plotly.graph_objects as go

*Sistema de archivos*

    ./
    
    ./Proyecto/
        ./Proyecto/Proyecto_bootcamp_cuaderno.ipynb

    ./Datasets/
        ./Datasets/...... .csv
    
    ./CleanDatasets/
        ./CleanDatasets/.....  .parquet


# Limpieza de archivos precipitaciones - conversión a parquet

### Primera exploración

Se importa el primer documento previo a la limpieza. Se inicia con el archivo de menor tamaño. Todos los datos corresponden a precipitación y tienen la misma unidad de medida.

In [ ]:
df = pd.read_csv('../Datasets/Precipitaci_n_20241016(vaupes).csv',
                 sep=',',
                 #nrows=10
                 )
df

En la siguiente celda se imprime un pequeño resumen. Es importante resaltar que no hay datos nulos y la necesidad de cambiar el formato de cada columna por uno mas adecuado que reduzca el tamaño del dataframe.

In [ ]:
size_0=df.memory_usage(deep=True).sum()
df.info()

### Vaupes

Se importa nuevamente el archivo csv, seleccionando las columnas relevantes y eligiendo un formato adeacuado para cada columna.

In [ ]:
df = pd.read_csv('../Datasets/Precipitaci_n_20241016(vaupes).csv',
                 sep=',',
                 usecols= [0,1,2,3,4,5,6,7,8,9],
                 dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category'}
                 )
df

In [ ]:
df.info()

Se muestra una comparación entre el tamaño del dataframe tras el cambio de formato a las columnas

In [ ]:
size_f=df.memory_usage(deep=True).sum()
print(size_f/size_0)

**Formato de fecha**

In [33]:
df['FechaObservacion'] = pd.to_datetime(df['FechaObservacion'], format='%m/%d/%Y %H:%M:%S %p')

In [ ]:
df.info()

In [35]:
df.to_parquet('../CleanDatasets/rain_vaupes.parquet')

In [ ]:
namesFiles = os.listdir('../Datasets')
namesFiles = [re.findall(r'\((\w*)\)',x)[0] for x in namesFiles]
print(namesFiles)

### Caquetá

Tener en cuenta que la información sobre caquetá está almacenada en dos archivos

#### Caquetá 2

In [ ]:
nameFile = '../Datasets/Precipitaci_n_20241016('+ namesFiles[13] +').csv'
nameFile

In [ ]:
df_caqueta2 = pd.read_csv(nameFile,
                 sep=',',
                 usecols= [0,1,2,3,4,5,6,7,8,9],
                 dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category'}
                 )
df_caqueta2['FechaObservacion'] = pd.to_datetime(df_caqueta2['FechaObservacion'], format='%m/%d/%Y %H:%M:%S %p')
df_caqueta2

La siguiente celda imprimirá la cantidad de valores de nulos

In [ ]:
nulls = df_caqueta2.count(axis=1).sum()
size = df_caqueta2.shape[0]*df_caqueta2.shape[1]
print(size-nulls)

De nuevo se observa que el archivo tiene un tamaño menor y que los diferentes formatos son aplicados correctamente a cada columna

In [ ]:
df_caqueta2.info()
#df_caqueta2.describe()

### Todos los departamentos

In [ ]:
print(namesFiles)
print(len(namesFiles))

In [42]:
for name in namesFiles[:]:
    nameFile = '../Datasets/Precipitaci_n_20241016('+ name +').csv'
    df_dept = pd.read_csv(nameFile,
                 sep=',',
                 usecols= [0,1,2,3,4,5,6,7,8,9],
                 dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category'}
                 )
    df_dept['FechaObservacion'] = pd.to_datetime(df_dept['FechaObservacion'], format='%m/%d/%Y %H:%M:%S %p')
    nulls = df_dept.count(axis=1).sum()
    size = df_dept.shape[0]*df_dept.shape[1]
    if nulls-size != 0:
        print(f'{nulls-size} nulls in {name}, parquet file can not created')
    else:
        df_dept.to_parquet(f'../CleanDatasets/rain_{name}.parquet')


# Consolidación de archivos

Se requiere un conjunto de datos que contiene la información de las estaciones de monitoreo.

A continuación se listan los archivos de la segunda carpeta que contiene los datasets:

    - Catálogo nacional de estaciones (Contine el listado de las estaciones)
    - Datos de estaciones IDEAM y terceros (Contiene mediciones de diversas estaciones propias y externas al IDEAM)
    - Datos hidrometeorológicos (Mediciones de temperatura)
    - Presión atmosférica  

In [ ]:
namesFiles2 = os.listdir('../Datasets2')
namesFiles2 = [x for x in namesFiles2]
print(namesFiles2)

### Limpieza *DATOS ESTACIONES IDEAM*

La ruta de acceso al archivo de Datos estaciones IDEAM es f'../Datasets2/{namesFiles2[1]}'

In [ ]:
df_estaciones = pd.read_csv(f'../Datasets2/{namesFiles2[1]}',
                            sep=',',
                            nrows=3
)
df_estaciones

In [ ]:
df_estaciones = pd.read_csv(f'../Datasets2/{namesFiles2[1]}',
                            sep=',',
                            dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category',10:'category',11:'category',12:'category'}
                            #nrows=3
)
df_estaciones.info()

Se evidencia que no hay valores nulos. Hacemos una conversión a formato de fecha y almacenamos el archivo en formato .parquet

In [ ]:
df_estaciones['FechaObservacion'] = pd.to_datetime(df_estaciones['FechaObservacion'], format='%m/%d/%Y %H:%M:%S %p')
df_estaciones.info()

In [24]:
df_estaciones.to_parquet('../CleanDatasets2/estaciones.parquet')

### Limpieza *DATOS PRESIÓN ATMOSFÉRICA*

In [ ]:
namesFiles2

La ruta de acceso a los datos de presión atmosférica es: f '../Datasets2/{namesFiles2[-1]}']

In [ ]:
df_presion = pd.read_csv(
                        f'../Datasets2/{namesFiles2[-1]}',
                        sep=',',
                        nrows=3
)
df_presion

In [ ]:
df_presion = pd.read_csv(
                        f'../Datasets2/{namesFiles2[-1]}',
                        sep=',',
                        dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category',10:'category',11:'category'}
)
df_presion.info()

In [ ]:
df_presion['FechaObservacion'] = pd.to_datetime(df_presion['FechaObservacion'],format='%m/%d/%Y %H:%M:%S %p')
df_presion

In [ ]:
df_presion.count()

Se observa con la función .count() que el tamaño de valores no nulos es igual al índice del rango del dataframe encontrado en la celda anterior.

In [33]:
df_presion.to_parquet('../CleanDatasets2/presion.parquet')

### Limpieza *DATOS TEMPERATURA*

Los datos de temperatura se distribuyen en 5 archivos csv distintos, todos con las mismas columnas. La ruta de acceso a los archivos es 

- '../Datasets2/Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(*)' Donde * es un valor entre 0 y 4

In [ ]:
namesFiles2[2:-1]

In [ ]:
df_temp = pd.read_csv(
                        f'../Datasets2/{namesFiles2[2]}',
                        sep=',',
                        nrows=3
)
df_temp

In [ ]:
df_temp = pd.read_csv(
                        f'../Datasets2/{namesFiles2[2]}',
                        sep=',',
                        dtype= {0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category',10:'category',11:'category'}         
                    )
df_temp

In [ ]:
df_temp.info()

In [ ]:
df_temp['FechaObservacion'] = pd.to_datetime(df_temp['FechaObservacion'], format='%m/%d/%Y %H:%M:%S %p')
df_temp.info()

In [51]:
df_temp.to_parquet('../CleanDatasets2/Temp_0.parquet')

In [8]:
index_temp = 1
for i in namesFiles2[3:-1]:
    df_temp = pd.read_csv(
                        f'../Datasets2/{i}',
                        sep=',',
                        dtype= {0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category',10:'category',11:'category'}         
                    )
    df_temp['FechaObservacion'] = pd.to_datetime(df_temp['FechaObservacion'], format='%m/%d/%Y %H:%M:%S %p')
    df_temp.to_parquet(f'../CleanDatasets2/Temp_{str(index_temp)}.parquet')
    index_temp += 1

# Análisis exploratorio de datos

In [ ]:
df_2 = pd.read_parquet(f'../CleanDatasets/rain_boyaca.parquet')
df_2

In [ ]:
df_2.info()

In [ ]:
df_2['Year'] = df_2['FechaObservacion'].dt.year
df_2['Month'] = df_2['FechaObservacion'].dt.month
year_total = (
    df_2
    .groupby(["Year"])["ValorObservado"]
    .sum()
    .reset_index()
)
year_total

In [ ]:
fig = make_subplots(rows=1, cols=1
                    #  , row_heights =[0.7, 0.3]
                    # , shared_xaxes = True
                    ,subplot_titles = ['Año'])

fig.add_trace(
    go.Scatter(x=year_total["Year"], y = year_total["ValorObservado"]
               , mode='lines+markers' # selcciona una combinación entre líneas y puntos
               , name = "Observado"
               ,legendgroup = '1'
               ),
)
###### Si sale error, toca instalar nbformat

Los archivos en formato parquet pueden ser consultados en la carpeta compartida de google drive: https://drive.google.com/drive/folders/1h9_VFIFKbZ8tiavxZM9Nm9koIxJxtCWf?usp=sharing

Se puede acceder a la carpeta compartida desde local instalando la versión de escritorio y buscando la ruta de los archivos.

In [ ]:
namesFiles = os.listdir(r'G:\Mi unidad\Bootcamp\Proyecto_Bootcamp\Data\CleanDatasets')
print('Primeros 23')
print(namesFiles[0:23])
print('Últimos 23')
print(namesFiles[23:-1])

CleanDatasets contiene los archivos parquet sobre precipitaciones

CleanDatasets2 contiene los archivos parquet sobre temperatura, presión, catálogo de estaciones.

Configuramos rutas para trabajar en local.

In [13]:
path = r'G:\Mi unidad\Bootcamp\Proyecto_Bootcamp\Data'                      

In [4]:
path = r'C:\Users\Josue Florez\Documents\Maria Angelica\Proyectos\Proyecto_bootcamp\Data'

## 23 primeros archivos de precipitación

In [5]:
folder_path = path

files = [f for f in os.listdir(folder_path)]

for file in files:
    file_path = os.path.join(folder_path, file)
    
    
    df = pd.read_parquet(file_path)
    rows, cols = df.shape
    
    print(f"Archivo: {file} - Filas: {rows}, Columnas: {cols}")


Archivo: estaciones.parquet - Filas: 202642, Columnas: 13
Archivo: rain_amazonas.parquet - Filas: 706717, Columnas: 10
Archivo: rain_antioquia.parquet - Filas: 14178669, Columnas: 10
Archivo: rain_arauca.parquet - Filas: 794950, Columnas: 10
Archivo: rain_atlantico1.parquet - Filas: 1371211, Columnas: 10
Archivo: rain_atlantico2.parquet - Filas: 1156367, Columnas: 10
Archivo: rain_bogota1.parquet - Filas: 1779933, Columnas: 10
Archivo: rain_bogota2.parquet - Filas: 19408968, Columnas: 10
Archivo: rain_bogota3.parquet - Filas: 3180034, Columnas: 10
Archivo: rain_bolivar1.parquet - Filas: 1917368, Columnas: 10
Archivo: rain_bolivar2.parquet - Filas: 1498134, Columnas: 10
Archivo: rain_boyaca.parquet - Filas: 13730170, Columnas: 10
Archivo: rain_caldas.parquet - Filas: 9882585, Columnas: 10
Archivo: rain_caqueta1.parquet - Filas: 1312815, Columnas: 10
Archivo: rain_caqueta2.parquet - Filas: 14546, Columnas: 10
Archivo: rain_casanare.parquet - Filas: 2615277, Columnas: 10
Archivo: rain_cau

In [7]:
for file in files:
    file_path = os.path.join(folder_path, file)
    
    df = pd.read_parquet(file_path)
    
    rows, cols = df.shape
    
    column_types = df.dtypes  
    
    print(f"Archivo: {file} - Filas: {rows}, Columnas: {cols}")
    print("Tipos de las columnas:")
    print(column_types)
    print("\n")

Archivo: estaciones.parquet - Filas: 202642, Columnas: 13
Tipos de las columnas:
CodigoEstacion             category
CodigoSensor               category
FechaObservacion     datetime64[ns]
ValorObservado              float32
NombreEstacion             category
Departamento               category
Municipio                  category
ZonaHidrografica           category
Latitud                    category
Longitud                   category
DescripcionSensor          category
UnidadMedida               category
Entidad                    category
dtype: object


Archivo: rain_amazonas.parquet - Filas: 706717, Columnas: 10
Tipos de las columnas:
CodigoEstacion            category
CodigoSensor              category
FechaObservacion    datetime64[ns]
ValorObservado             float32
NombreEstacion            category
Departamento              category
Municipio                 category
ZonaHidrografica          category
Latitud                   category
Longitud                  category
d

In [ ]:

fecha_columna = 'FechaObservacion' 

for file in files:
    file_path = os.path.join(folder_path, file)

    df = pd.read_parquet(file_path)
    
    rows, cols = df.shape
    
    if fecha_columna in df.columns:
       
        df[fecha_columna] = pd.to_datetime(df[fecha_columna], errors='coerce')

       
        fecha_minima = df[fecha_columna].min()
        fecha_maxima = df[fecha_columna].max()
        
        print(f"Archivo: {file} - Filas: {rows}, Columnas: {cols}")
        print(f"Fecha mínima: {fecha_minima}")
        print(f"Fecha máxima: {fecha_maxima}\n")
    else:
        print(f"Archivo: {file} - No se encontró la columna '{fecha_columna}'\n")


In [32]:
dataframes = {}

for file in files:
    file_path = os.path.join(folder_path, file)
    
    df = pd.read_parquet(file_path)
    
    nombre_ciudad = file.split('_')[0]  # Extraer el nombre de la ciudad del nombre del archivo
    dataframes[nombre_ciudad] = df

In [37]:
dataframes={}

In [35]:
resultados_agrupados = {}
for file in files:
    file_path = os.path.join(folder_path, file)
    
    df = pd.read_parquet(file_path)
    
    df['FechaObservacion'] = pd.to_datetime(df['FechaObservacion'], errors='coerce')
    
    
    df['Year'] = df['FechaObservacion'].dt.year
    df['Month'] = df['FechaObservacion'].dt.month
    
    
    df_grouped = df.groupby(['Year', 'Month','Departamento'])['ValorObservado'].agg(['sum', 'mean']).reset_index()
    
    df_grouped.columns = ['Año', 'Mes', 'Departamento','Valor Total', 'Valor Promedio']
    
    # Guardar los resultados en el diccionario
    resultados_agrupados[nombre_ciudad] = df_grouped

C:\Users\Josue Florez\AppData\Local\Temp\ipykernel_10844\949150212.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_grouped = df.groupby(['Year', 'Month','Departamento'])['ValorObservado'].agg(['sum', 'mean']).reset_index()
C:\Users\Josue Florez\AppData\Local\Temp\ipykernel_10844\949150212.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_grouped = df.groupby(['Year', 'Month','Departamento'])['ValorObservado'].agg(['sum', 'mean']).reset_index()
C:\Users\Josue Florez\AppData\Local\Temp\ipykernel_10844\949150212.py:14: FutureWarning: The default of observed=False is deprecated and will be changed t

In [39]:
# Llamar al DataFrame de Bogotá
df_bogota =  resultados_agrupados[nombre_ciudad]
df_bogota



,Año,Mes,Departamento,Valor Total,Valor Promedio
0,2017,1,GUAINÍA,0.000000,NaN
1,2017,2,GUAINÍA,13.400001,0.003826
2,2017,3,GUAINÍA,104.099998,0.029717
3,2017,4,GUAINÍA,0.000000,NaN
4,2017,5,GUAINÍA,0.000000,NaN
...,...,...,...,...,...
91,2024,8,GUAINÍA,261.600006,0.058906
92,2024,9,GUAINÍA,263.100006,0.062287
93,2024,10,GUAINÍA,68.199997,0.016757
94,2024,11,GUAINÍA,0.000000,NaN


# EDA Bogota2

In [11]:
file_bogota=path+r'\rain_bogota2.parquet'
df_bogota = pd.read_parquet(file_bogota)

In [ ]:
df_bogota.describe()

In [ ]:
df_bogota.describe(include=['category'])

In [ ]:

print(df_bogota['Departamento'].value_counts())
print(df_bogota['NombreEstacion'].value_counts())


In [ ]:
df_bogota['Year'] = df_bogota['FechaObservacion'].dt.year
df_bogota['Month'] = df_bogota['FechaObservacion'].dt.month
year_total = (
    df_bogota
    .groupby(["Year"])["ValorObservado"]
    .sum()
    .reset_index()
)
year_total

In [ ]:
year_mean = (
    df_bogota
    .groupby(["Year"])["ValorObservado"]
    .mean()
    .reset_index()
)
year_mean

In [ ]:
fig = make_subplots(rows=1, cols=1
                    #  , row_heights =[0.7, 0.3]
                    # , shared_xaxes = True
                    ,subplot_titles = ['Año'])
fig.add_trace(
    go.Scatter(x=year_total["Year"], y = year_total["ValorObservado"]
               , mode='lines+markers' # selcciona una combinación entre líneas y puntos
               , name = "Observado"
               ,legendgroup = '1'
               ),
)

In [ ]:
month_total = (
    df_bogota
    .groupby(["Month"])["ValorObservado"]
    .sum()
    .reset_index()
)
month_total

In [ ]:
fig = make_subplots(rows=1, cols=1
                    #  , row_heights =[0.7, 0.3]
                    # , shared_xaxes = True
                    ,subplot_titles = ['Meses'])
fig.add_trace(
    go.Scatter(x=month_total["Month"], y = month_total["ValorObservado"]
               , mode='lines+markers' # selcciona una combinación entre líneas y puntos
               , name = "Observado"
               ,legendgroup = '1'
               ),
)

In [ ]:
df_bogota['Year'] = df_bogota['FechaObservacion'].dt.year
df_bogota['Month'] = df_bogota['FechaObservacion'].dt.month
df_grouped = df_bogota.groupby(['Year', 'Month'])['ValorObservado'].agg(['sum', 'mean']).reset_index()

df_grouped.columns = ['Year', 'Month', 'Valor Total', 'Valor Promedio']

df_grouped

In [ ]:
df_grouped.max()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='Month', y='Valor Total', data=df_grouped, marker='o')
plt.title('Distribución del Valor Total Observado por Mes')
plt.xlabel('Mes')
plt.ylabel('Valor Total')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='Month', y='Valor Promedio', data=df_grouped, marker='o')
plt.title('Distribución del Valor Promedio Observado por Mes')
plt.xlabel('Mes')
plt.ylabel('Valor Promedio')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='Year', y='Valor Total', data=df_grouped, palette='Blues_d')
plt.title('Distribución del Valor Total Observado por Año')
plt.xlabel('Año')
plt.ylabel('Valor Total')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='Year', y='Valor Promedio', data=df_grouped)
plt.title('Distribución del Valor Promedio Observado por Año')
plt.xlabel('Año')
plt.ylabel('Valor Promedio')
plt.show()

In [ ]:
df_pivot = df_grouped.pivot(index='Year', columns='Month', values='Valor Total')

# Heatmap para visualizar el valor total observado por año y mes
plt.figure(figsize=(12, 6))
sns.heatmap(df_pivot, cmap='coolwarm',annot=True,fmt='.0f')
plt.title('Distribución del Valor Total Observado por Año y Mes')
plt.show()

In [ ]:
df_grouped = df_bogota.groupby(['Year', 'Month','Latitud','Longitud'])['ValorObservado'].agg(['sum', 'mean']).reset_index()

df_grouped.columns = ['Year', 'Month','Latitud','Longitud', 'Valor Total', 'Valor Promedio']

df_grouped

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Latitud', y='Longitud', hue='Valor Total', data=df_grouped, palette='coolwarm')
plt.title('Distribución geográfica de Valor Observado')
plt.show

## 23 últimos archivos de precipitación

Lista de archivos:

In [ ]:
namesFiles = os.listdir(path+r'\CleanDatasets')[23:-1]
print(namesFiles)

### Dimensionalidad de los dataset

In [ ]:
dim = pd.DataFrame(columns=['File','Rows','Columns','Size','Nulls'])

for i in namesFiles:
    path_file = path + '\\CleanDatasets\\' + i
    df_dept2 = pd.read_parquet(
                                path_file
                            )
    row = pd.DataFrame({'File':[i],'Rows':[df_dept2.shape[0]],'Columns':[df_dept2.shape[1]],'Size':[df_dept2.size],'Nulls':[df_dept2.size-df_dept2.count().sum()]})
    dim = pd.concat([dim,row], ignore_index=True)

dim

Tipos de datos

In [ ]:
for i in namesFiles:
    path_file = path + '\\CleanDatasets\\' + i
    df_dept2 = pd.read_parquet(
                                path_file
                            )
    columns = df_dept2.dtypes.to_dict()
    row = pd.DataFrame(columns, index=)
    dim = pd.concat([dim,row], ignore_index=True)

types

In [ ]:
df_dept2.dtypes.to_dict()